What is the average priced paid per amenity
What is the average price paid per top 5 combined amenities
Can we predict what the top amenities are or what properties are being sold based on amenities
Why is TV missing from top of list.  -- Manually checked if most properties have TV

In [1]:
import pandas as pd
import nltk
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from ast import literal_eval

import numpy as np
# nltk.download('punkt')

# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_row',None)

In [2]:
Boston = pd.read_csv('../Data_Files/BostonGZ_listings.csv')
df2 = pd.read_csv('../Data_Files/Boston_listings.csv')

In [3]:
##Due to file type being csv, the column is converted to a string automatically.
##literal_eval is used to convert string to list format.

Boston_amenities_list = []
for items in Boston.amenities:
    Boston_amenities_list.append(literal_eval(items))

Boston_amenities_list[34]
##Count to see if TV in list

##TV should be the estimated number one amenity
##Code to combine TV and put in list.
updated_boston_amenities = []
Boston_amenities_list = [['TV' if 'TV' in item else item for item in lst] for lst in Boston_amenities_list ]

#need to account for duplicate TV in lists or other duplicates
#len(Boston_amenities_list)
Bost_amen_list=[]
for l in Boston_amenities_list:
    Bost_amen_list.append([*set(l)])

In [4]:
##20 Most Common Amenities
##Why is TV missing
## Assumption was made that TV was going to be number one.
##After some data wrangling TV is estimated at number 9

result_list = []
amenities_counter = Counter()

for amens in Bost_amen_list:
    amenities_counter.update(amens)

In [5]:
#top combined amenities
n = 4
n_gram_counter = Counter()
for gram in Bost_amen_list:
    gram_list = nltk.ngrams(gram,n)
    n_gram_counter.update(gram_list)

In [6]:
# amenities_counter.most_common(100)
top_100_amenities=[word for word,cnt in amenities_counter.most_common(100)]
df = pd.DataFrame(columns=top_100_amenities)

zeros = [0]* 100

counter = 0

for element in Bost_amen_list:
    df.loc[-1]= zeros
    df = df.reset_index(drop=True)
    for col in df.columns:
        for item in element:
            if item == col:
                df.at[counter,col] =1
    counter+=1          

In [7]:
# ##Set of all unique amenities.
# amenities = list(Boston.amenities)
# amenities_string = " ".join(amenities)
# amenities_string = amenities_string.replace('{', '')
# amenities_string = amenities_string.replace('}', ',')
# amenities_string = amenities_string.replace('[', '')
# amenities_string = amenities_string.replace(']', ',')
# amenities_string = amenities_string.replace('"', '')
# amenities_set = [x.strip() for x in amenities_string.split(',')]
# # amenities_set = set(amenities_set)
# # amenities_set

# amenities_set

In [8]:
Boston.head()
Boston_id_price = Boston[["host_id","price"]]

In [9]:
Boston_concat_price_amenites = pd.concat([Boston_id_price, df], axis=1)

In [10]:
#Convert dollar amounts to floats
Boston_concat_price_amenites['price']  = Boston_concat_price_amenites['price'].replace('[\$,]', '', regex=True).astype(float)

#test average or median
median = round(Boston_concat_price_amenites.loc[Boston_concat_price_amenites['Wifi'] == 1, 'price'].median(),2)
mean = round(Boston_concat_price_amenites.loc[Boston_concat_price_amenites['Wifi'] == 1, 'price'].mean(),2)

In [11]:
#This cell cannot be ran more than once or it inserts an extra row.

average_price=[]
for col in df.columns:
    average_price.append(round(Boston_concat_price_amenites.loc[Boston_concat_price_amenites[col] == 1, 'price'].mean(),2))

average_price.insert(0,float("nan"))
average_price.insert(0,float("nan"))

average_price

Boston_concat_price_amenites.loc[-1] = average_price  # adding a row
Boston_concat_price_amenites.index = Boston_concat_price_amenites.index + 1  # shifting index
Boston_concat_price_amenites = Boston_concat_price_amenites.sort_index()